In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
os.chdir('/home/bohdan/projects/glider/lux-ai')

In [4]:
from rl_agents.agent import Agent
from agents.lux.config import EnvConfig
from luxai_s2.env import LuxAI_S2
import numpy as np

configurations = {'max_episode_length': 1000, 'map_size': 48, 'verbose': 2, 'validate_action_space': True, 'max_transfer_amount': 3000, 'MIN_FACTORIES': 2, 'MAX_FACTORIES': 5, 'CYCLE_LENGTH': 50, 'DAY_LENGTH': 30, 'UNIT_ACTION_QUEUE_SIZE': 20, 'MAX_RUBBLE': 100, 'FACTORY_RUBBLE_AFTER_DESTRUCTION': 50, 'INIT_WATER_METAL_PER_FACTORY': 150, 'INIT_POWER_PER_FACTORY': 1000, 'MIN_LICHEN_TO_SPREAD': 20, 'LICHEN_LOST_WITHOUT_WATER': 1, 'LICHEN_GAINED_WITH_WATER': 1, 'MAX_LICHEN_PER_TILE': 100, 'POWER_PER_CONNECTED_LICHEN_TILE': 1, 'LICHEN_WATERING_COST_FACTOR': 10, 'BIDDING_SYSTEM': True, 'FACTORY_PROCESSING_RATE_WATER': 100, 'ICE_WATER_RATIO': 4, 'FACTORY_PROCESSING_RATE_METAL': 50, 'ORE_METAL_RATIO': 5, 'FACTORY_CHARGE': 50, 'FACTORY_WATER_CONSUMPTION': 1, 'POWER_LOSS_FACTOR': 0.5, 'ROBOTS': {'LIGHT': {'METAL_COST': 10, 'POWER_COST': 50, 'CARGO_SPACE': 100, 'BATTERY_CAPACITY': 150, 'CHARGE': 1, 'INIT_POWER': 50, 'MOVE_COST': 1, 'RUBBLE_MOVEMENT_COST': 0.05, 'DIG_COST': 5, 'DIG_RUBBLE_REMOVED': 2, 'DIG_RESOURCE_GAIN': 2, 'DIG_LICHEN_REMOVED': 10, 'SELF_DESTRUCT_COST': 10, 'RUBBLE_AFTER_DESTRUCTION': 1, 'ACTION_QUEUE_POWER_COST': 1}, 'HEAVY': {'METAL_COST': 100, 'POWER_COST': 500, 'CARGO_SPACE': 1000, 'BATTERY_CAPACITY': 3000, 'CHARGE': 10, 'INIT_POWER': 500, 'MOVE_COST': 20, 'RUBBLE_MOVEMENT_COST': 1, 'DIG_COST': 60, 'DIG_RUBBLE_REMOVED': 20, 'DIG_RESOURCE_GAIN': 20, 'DIG_LICHEN_REMOVED': 100, 'SELF_DESTRUCT_COST': 100, 'RUBBLE_AFTER_DESTRUCTION': 10, 'ACTION_QUEUE_POWER_COST': 10}}}
agent = Agent('player_0', configurations)

env_config = EnvConfig.from_dict(configurations)

pygame 2.2.0 (SDL 2.0.22, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
def interact(env, agents, steps):
    # reset our env
    obs = env.reset(seed=41)
    np.random.seed(0)
    imgs = []
    step = 0
    # Note that as the environment has two phases, we also keep track a value called 
    # `real_env_steps` in the environment state. The first phase ends once `real_env_steps` is 0 and used below

    # iterate until phase 1 ends
    while env.state.real_env_steps < 0:
        if step >= steps: break
        actions = {}
        for player in env.agents:
            o = obs[player]
            a = agents[player].early_setup(step, o)
            actions[player] = a
        step += 1
        obs, rewards, dones, infos = env.step(actions)
        imgs += [env.render("rgb_array", width=640, height=640)]
    done = False
    while not done:
        if step >= steps: break
        actions = {}
        for player in env.agents:
            o = obs[player]
            a = agents[player].act(step, o)
            actions[player] = a
        step += 1
        obs, rewards, dones, infos = env.step(actions)
        imgs += [env.render("rgb_array", width=640, height=640)]
        done = dones["player_0"] and dones["player_1"]

In [23]:
env = LuxAI_S2()
obs = env.reset(42)
agents = {player: Agent(player, env.state.env_cfg) for player in env.agents}
interact(env, agents, 1000)